### Get start with iris dataset

In [58]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf
import pandas as pd

In [60]:
## Load iris dataset

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

def load_data(label_name='Species'):
    """Parses the csv file in TRAIN_URL and TEST_URL."""

    train_path = tf.keras.utils.get_file(fname=TRAIN_URL.split('/')[-1],
                                         origin=TRAIN_URL)
    # train_path now holds the pathname: ~/.keras/datasets/iris_training.csv

    train = pd.read_csv(filepath_or_buffer=train_path,
                        names=CSV_COLUMN_NAMES,
                        header=0)
    train_features, train_label = train, train.pop(label_name)

    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
    test = pd.read_csv(test_path, 
                       names=CSV_COLUMN_NAMES, 
                       header=0)
    test_features, test_label = test, test.pop(label_name)


    return (train_features, train_label), (test_features, test_label)

(train_feature, train_label), (test_feature, test_label) = load_data()

In [61]:
## Classifier

my_feature_columns = [tf.feature_column.numeric_column(key=key) 
                      for key in train_feature.keys()]

classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=[10, 10],
        n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x181a5a6450>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/9r/rtf8mwj90551dp6rwnn9j8xr0000gn/T/tmpgFXE6J', '_save_summary_steps': 100}


In [62]:
## Train the model

def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

classifier.train(
        input_fn=lambda:train_input_fn(train_feature, train_label, 100),
        steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/9r/rtf8mwj90551dp6rwnn9j8xr0000gn/T/tmpgFXE6J/model.ckpt.
INFO:tensorflow:loss = 212.84041, step = 1
INFO:tensorflow:global_step/sec: 543.815
INFO:tensorflow:loss = 28.603666, step = 101 (0.186 sec)
INFO:tensorflow:global_step/sec: 710.071
INFO:tensorflow:loss = 17.038551, step = 201 (0.141 sec)
INFO:tensorflow:global_step/sec: 697.628
INFO:tensorflow:loss = 11.513592, step = 301 (0.143 sec)
INFO:tensorflow:global_step/sec: 545.569
INFO:tensorflow:loss = 8.5943165, step = 401 (0.184 sec)
INFO:tensorflow:global_step/sec: 560.045
INFO:tensorflow:loss = 7.9833302, step = 501 (0.179 sec)
INFO:tensorflow:global_step/sec: 647.128
INFO:tensorflow:loss = 6.145497, step = 601 (0.154 sec)
INFO:tensorflow:global_step/sec: 540.605
INFO:tensorflow:loss = 7.02843, step = 701 (0.187 sec)
INFO:tensorflow:global_step/sec: 530.105
INFO:tensorflow:loss = 4.398101, step = 801 (0.187 sec)
INFO:tensorflow:

In [50]:
## Evaluate the model

def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    if labels is None:
        inputs = dict(features) # input should be a dictionary
    else:
        inputs = (dict(features), labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_feature, test_label, 10))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-04-18-12:04:30
INFO:tensorflow:Restoring parameters from /var/folders/9r/rtf8mwj90551dp6rwnn9j8xr0000gn/T/tmpcppwOo/model.ckpt-1100
INFO:tensorflow:Finished evaluation at 2018-04-18-12:04:31
INFO:tensorflow:Saving dict for global step 1100: accuracy = 0.96666664, average_loss = 0.057569515, global_step = 1100, loss = 0.57569516

Test set accuracy: 0.967

